# Readme

Sometimes recordings stop working after some time and the recordings stacks contain pure noise after a certain time.

In [ ]:
import datajoint as dj
import os

In [ ]:
username = !whoami
username = username[0]
username

In [ ]:
home_directory = os.path.expanduser("~")
home_directory

In [ ]:
# Set config file
config_file = f'{home_directory}/datajoint/dj_{username}_conf.json'
assert os.path.isfile(config_file), f'Set the path to your config file: {config_file}'

In [ ]:
dj.config.load(config_file)

In [ ]:
schema_name = "ageuler_ENTER_SCHEMA_NAME_HERE"
    
schema = dj.schema(schema_name)
schema

In [ ]:
schema.spawn_missing_classes()  # This should spawn Presentation

In [ ]:
h5_headers = Presentation().fetch('h5_header')

# Find corrupted recordings

In [ ]:
import numpy as np
from djimaging.utils import scanm_utils
from tqdm.notebook import tqdm
from matplotlib import pyplot as plt


variances = []
for h5_file in tqdm(h5_headers):
    ch_stacks, wparams = scanm_utils.load_stacks_from_h5(h5_file)
    variances.append(np.var(ch_stacks['wDataCh0'], axis=(0, 1)))
variances = np.array(variances, dtype=object)

def fun_badness_metric(var):
    return np.minimum(np.nanmean(var[100:200]) / (np.maximum(1e-9, np.nanmean(var[-200:]))), 10)

badness_metrics = np.array([fun_badness_metric(var) for var in variances])
out_i = badness_metrics > np.mean(badness_metrics) + 3 * np.std(badness_metrics)

plt.figure()
plt.hist(badness_metrics[~out_i], fc='b', bins=np.linspace(np.min(badness_metrics), np.max(badness_metrics), 51))
plt.hist(badness_metrics[out_i], fc='r', bins=np.linspace(np.min(badness_metrics), np.max(badness_metrics), 51))

print(h5_headers[out_i])

# Plot corrupted recordings

In [ ]:
fig, axs = plt.subplots(np.sum(out_i), 4, figsize=(12, np.sum(out_i)*2))

for i, h5_file in enumerate(h5_headers[out_i]):
    ch_stacks, wparams = scanm_utils.load_stacks_from_h5(h5_file)
    var_i = np.var(ch_stacks['wDataCh0'], axis=(0, 1))
    bad_idx = np.argmax(var_i < 100) - 2
    print('(\"', h5_file, '\", ', bad_idx, ')', sep='')
    
    im0 = np.nanmean(ch_stacks['wDataCh0'][:, :, 100:200], axis=2)
    im1 = np.nanmean(ch_stacks['wDataCh0'][:, :, -100:], axis=2)
    im2 = ch_stacks['wDataCh0'][:, :, bad_idx-1]

    for j, imj in enumerate([im0, im1, im2]):
        ax = axs[i, j]
        im = ax.imshow(imj.T)
        plt.colorbar(im, ax=ax)

    ax = axs[i, -1]
    ax.plot(variances)
    ax.axvline(bad_idx, c='r')

plt.tight_layout()